In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../BigQuery/letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()

# Clean track info
Only Retain the information we need.
- id
- name
- explicit : 1 for explicit lyrics and 0 for not
- available_markets : number of available markets
- popularity

In [3]:
track_info = pd.read_csv("../Audio Features & Track/tracks info.csv")

In [13]:
def compute_markets(s):
    markets = s.replace('[','').replace(']','').replace("'",'').replace(' ', '').split(',')
    if markets[0] == '':
        return 0
    else:
        return len(markets)

In [14]:
cat = pd.unique(track_info['explicit'])
track_info['num_markets'] = track_info['available_markets'].apply(compute_markets)
track_info['explicit_trans'] = None 
track_info['explicit_trans'][track_info['explicit'] == cat[0]] = 0
track_info['explicit_trans'][track_info['explicit'] == cat[1]] = 1


In [15]:
track_info.head()

,Unnamed: 0,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri,num_markets,explicit_trans
0,0,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,160106,False,{'isrc': 'QZ7B41700002'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/001CyR8xqmmp...,001CyR8xqmmpVZFiTZJ5BC,False,She Knows How To Rock Me,30,NaN,2,track,spotify:track:001CyR8xqmmpVZFiTZJ5BC,184,0
1,1,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",3,602266,False,{'isrc': 'DEF057630640'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/001eyxfoYptA...,001eyxfoYptAWzvF4ewLrR,False,"Die Meistersinger von Nürnberg, WWV 96 / Act I...",22,NaN,7,track,spotify:track:001eyxfoYptAWzvF4ewLrR,184,0
2,2,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,212826,False,{'isrc': 'GBUM72107273'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/002LxxV7VTy5...,002LxxV7VTy5JkvkmmGanl,False,This Foolish Heart Could Love You,42,NaN,1,track,spotify:track:002LxxV7VTy5JkvkmmGanl,183,0
3,3,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,304933,False,{'isrc': 'GBAYE0300927'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/002YdWxaenIU...,002YdWxaenIUL4PHiv5EB5,False,Eccentric Man - 2003 Remastered Version,31,https://p.scdn.co/mp3-preview/2c9eac1675b53cb6...,9,track,spotify:track:002YdWxaenIUL4PHiv5EB5,183,0
4,4,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,191226,False,{'isrc': 'QZMHL2231434'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/002nFT2kRsfv...,002nFT2kRsfvS0d17AwWsB,False,A Month Or Two,42,https://p.scdn.co/mp3-preview/02503d79c16a90fc...,1,track,spotify:track:002nFT2kRsfvS0d17AwWsB,185,0


In [16]:
import ast
track_info['album_id'] = track_info['album'].apply(lambda x: ast.literal_eval(x)['id'])

In [17]:
track_info['artist_id'] = track_info['artists'].apply(lambda x: ';'.join([artist['id'] for artist in ast.literal_eval(x)]))

In [18]:
track_info = track_info[['id','name',"album_id","artist_id",'popularity','explicit_trans','num_markets']]
track_info.rename(columns={'num_markets':'available_markets', 'explicit_trans':'explicit'}, inplace=True)

In [19]:
from datetime import date
track_info['date'] = date.today()

In [20]:
track_info.to_csv("track clear info.csv")

# Load track clear info to big query

In [22]:
dataset_id = "snappy-boulder-378707.TrackClearInfo"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, timeout=30)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset snappy-boulder-378707.TrackClearInfo


In [23]:
schema = [
    bigquery.SchemaField("unnamed","INTEGER"),
    bigquery.SchemaField("id","STRING"),
    bigquery.SchemaField("name","STRING"),
    bigquery.SchemaField("album_id","STRING"),
    bigquery.SchemaField("artist_id","STRING"),
    bigquery.SchemaField("popularity","INTEGER"),
    bigquery.SchemaField("explicit","INTEGER"),
    bigquery.SchemaField("available_markets","INTEGER"),
    bigquery.SchemaField("extract_date","DATE")
]


# TODO(developer): Set table_id to the ID of the table to create.
table_id = "snappy-boulder-378707.TrackClearInfo.TrackClearInfo"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, timeout=30)  # Make an API request.

In [24]:
job_config = bigquery.LoadJobConfig(
    schema = schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

file_path = './track clear info.csv'
table_id = "snappy-boulder-378707.TrackClearInfo.TrackClearInfo"

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 74925 rows and 9 columns to snappy-boulder-378707.TrackClearInfo.TrackClearInfo


In [25]:
%%bigquery
ALTER TABLE snappy-boulder-378707.TrackClearInfo.TrackClearInfo
DROP COLUMN unnamed

Query is running:   0%|          |

""


In [26]:
%%bigquery
ALTER TABLE snappy-boulder-378707.TrackClearInfo.TrackClearInfo
ADD PRIMARY KEY (id) NOT ENFORCED

Query is running:   0%|          |

""


# Clear Audio features

In [25]:

schema = [
    bigquery.SchemaField("unnamed","INTEGER"),
    bigquery.SchemaField("danceability","FLOAT"),
    bigquery.SchemaField("energy","FLOAT"),
    bigquery.SchemaField("key","INTEGER"),
    bigquery.SchemaField("loudness","FLOAT"),
    bigquery.SchemaField("mode","INTEGER"),
    bigquery.SchemaField("speechiness","FLOAT"),
    bigquery.SchemaField("acousticness","FLOAT"),
    bigquery.SchemaField("instrumentalness","FLOAT"),
    bigquery.SchemaField("liveness","FLOAT"),
    bigquery.SchemaField("valence","FLOAT"),
    bigquery.SchemaField("tempo","FLOAT"),
    bigquery.SchemaField("type","STRING"),
    bigquery.SchemaField("id","STRING"),
    bigquery.SchemaField("uri","STRING"),
    bigquery.SchemaField("track_href","STRING"),
    bigquery.SchemaField("analysis_url","STRING"),
    bigquery.SchemaField("duration_ms","INTEGER"),
    bigquery.SchemaField("time_signature","INTEGER"),
]

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "snappy-boulder-378707.AudioFeatures.AudioFeatures"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, timeout=30)  # Make an API request.
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Created table snappy-boulder-378707.AudioFeatures.AudioFeatures


In [26]:
job_config = bigquery.LoadJobConfig(
    schema = schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)


file_path = './audio features.csv'
table_id = "snappy-boulder-378707.AudioFeatures.AudioFeatures"

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 74856 rows and 19 columns to snappy-boulder-378707.AudioFeatures.AudioFeatures


In [34]:
%%bigquery
ALTER TABLE snappy-boulder-378707.AudioFeatures.AudioFeatures
DROP COLUMN unnamed, drop column type, drop column uri,drop column track_href,drop column analysis_url;

Query is running:   0%|          |

""


In [35]:
%%bigquery
ALTER TABLE snappy-boulder-378707.AudioFeatures.AudioFeatures
ADD PRIMARY KEY (id) NOT ENFORCED

Query is running:   0%|          |

""
